<a href="https://colab.research.google.com/github/IliaAltmark/spotify_classification/blob/main/jupyter_notebooks/Spotify_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setting Up

In [ ]:
!pip install spotipy

     |████████████████████████████████| 138 kB 7.5 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = "d2388538505141f7b0592d42b191e537" 
secret = "02a9e5c22e594342862a1d0a4a024964"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# 2. Get the Track ID Data

In [ ]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for year in range(2012,2022):

  for i in range(0,1000,50):
      track_results = sp.search(q=f'year:{year} genre:techno', type='track', limit=50,offset=i)
      for i, t in enumerate(track_results['tracks']['items']):
          artist_name.append(t['artists'][0]['name'])
          track_name.append(t['name'])
          track_id.append(t['id'])
          popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 50.614583683999996


# 3. EDA + Data Preparation

In [ ]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 10000


In [ ]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(10000, 4)


,artist_name,track_name,track_id,popularity
0,Zedd,Clarity,60wwxj6Dd9NJlirf84wr2c,74
1,Zedd,Stay The Night - Featuring Hayley Williams Of ...,2QtJA4gbwe1AcanB2p21aP,63
2,Zedd,Spectrum,1dFkD1JfRMzwO6hwUsE8aS,59
3,Zedd,Lost At Sea,3Gwq0zKnMWluOEPKh93BeC,47
4,Noir,Around - Solomun Vox,5qA35ZZWFOqynheP5x8rty,56


In [ ]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  10000 non-null  object
 1   track_name   10000 non-null  object
 2   track_id     10000 non-null  object
 3   popularity   10000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


In [ ]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

410

In [ ]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [ ]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [ ]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()


artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [ ]:
df_tracks.shape

(9547, 4)

# 4: Get the Audio Features Data

In [ ]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 18.810437000999997


# 5. EDA + Data Preparation

In [ ]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 9547


In [ ]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (9547, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.509,0.781,8,-3.480,1,0.0720,0.0398,0.00000,0.0749,0.176,128.000,audio_features,60wwxj6Dd9NJlirf84wr2c,spotify:track:60wwxj6Dd9NJlirf84wr2c,https://api.spotify.com/v1/tracks/60wwxj6Dd9NJ...,https://api.spotify.com/v1/audio-analysis/60ww...,271427,4
1,0.596,0.738,8,-3.109,1,0.0411,0.1090,0.00000,0.0947,0.461,127.961,audio_features,2QtJA4gbwe1AcanB2p21aP,spotify:track:2QtJA4gbwe1AcanB2p21aP,https://api.spotify.com/v1/tracks/2QtJA4gbwe1A...,https://api.spotify.com/v1/audio-analysis/2QtJ...,217347,4
2,0.609,0.801,0,-4.417,0,0.0340,0.0137,0.00349,0.1270,0.407,129.990,audio_features,1dFkD1JfRMzwO6hwUsE8aS,spotify:track:1dFkD1JfRMzwO6hwUsE8aS,https://api.spotify.com/v1/tracks/1dFkD1JfRMzw...,https://api.spotify.com/v1/audio-analysis/1dFk...,243413,4
3,0.567,0.796,6,-3.534,1,0.0364,0.0247,0.00000,0.1240,0.305,127.983,audio_features,3Gwq0zKnMWluOEPKh93BeC,spotify:track:3Gwq0zKnMWluOEPKh93BeC,https://api.spotify.com/v1/tracks/3Gwq0zKnMWlu...,https://api.spotify.com/v1/audio-analysis/3Gwq...,225840,4
4,0.867,0.345,1,-11.042,1,0.0768,0.0212,0.66600,0.1030,0.259,115.000,audio_features,5qA35ZZWFOqynheP5x8rty,spotify:track:5qA35ZZWFOqynheP5x8rty,https://api.spotify.com/v1/tracks/5qA35ZZWFOqy...,https://api.spotify.com/v1/audio-analysis/5qA3...,418594,4


In [ ]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9547 entries, 0 to 9546
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      9547 non-null   float64
 1   energy            9547 non-null   float64
 2   key               9547 non-null   int64  
 3   loudness          9547 non-null   float64
 4   mode              9547 non-null   int64  
 5   speechiness       9547 non-null   float64
 6   acousticness      9547 non-null   float64
 7   instrumentalness  9547 non-null   float64
 8   liveness          9547 non-null   float64
 9   valence           9547 non-null   float64
 10  tempo             9547 non-null   float64
 11  type              9547 non-null   object 
 12  id                9547 non-null   object 
 13  uri               9547 non-null   object 
 14  track_href        9547 non-null   object 
 15  analysis_url      9547 non-null   object 
 16  duration_ms       9547 non-null   int64  


In [ ]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(9547, 14)

In [ ]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (9547, 17)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Zedd,Clarity,60wwxj6Dd9NJlirf84wr2c,74,0.509,0.781,8,-3.480,1,0.0720,0.0398,0.00000,0.0749,0.176,128.000,271427,4
1,Zedd,Stay The Night - Featuring Hayley Williams Of ...,2QtJA4gbwe1AcanB2p21aP,63,0.596,0.738,8,-3.109,1,0.0411,0.1090,0.00000,0.0947,0.461,127.961,217347,4
2,Zedd,Spectrum,1dFkD1JfRMzwO6hwUsE8aS,59,0.609,0.801,0,-4.417,0,0.0340,0.0137,0.00349,0.1270,0.407,129.990,243413,4
3,Zedd,Lost At Sea,3Gwq0zKnMWluOEPKh93BeC,47,0.567,0.796,6,-3.534,1,0.0364,0.0247,0.00000,0.1240,0.305,127.983,225840,4
4,Noir,Around - Solomun Vox,5qA35ZZWFOqynheP5x8rty,56,0.867,0.345,1,-11.042,1,0.0768,0.0212,0.66600,0.1030,0.259,115.000,418594,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9547 entries, 0 to 9546
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       9547 non-null   object 
 1   track_name        9547 non-null   object 
 2   track_id          9547 non-null   object 
 3   popularity        9547 non-null   int64  
 4   danceability      9547 non-null   float64
 5   energy            9547 non-null   float64
 6   key               9547 non-null   int64  
 7   loudness          9547 non-null   float64
 8   mode              9547 non-null   int64  
 9   speechiness       9547 non-null   float64
 10  acousticness      9547 non-null   float64
 11  instrumentalness  9547 non-null   float64
 12  liveness          9547 non-null   float64
 13  valence           9547 non-null   float64
 14  tempo             9547 non-null   float64
 15  duration_ms       9547 non-null   int64  
 16  time_signature    9547 non-null   int64  


Checking for any duplicate tracks

In [ ]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


# Enriching data with additional features

In [ ]:
import time
from tqdm import tqdm

In [ ]:
# again measuring the time
start = timeit.default_timer()

# creating a dataset with additional features
track_id = []
key_confidence = []
mode_confidence = []
tempo_confidence = []
time_signature_confidence = []
None_counter = 0

for i in tqdm(range(0,len(df['track_id']))):
    if i % 100 == 0:
      time.sleep(2)
      
    track = df['track_id'][i]

    try:
      analysis_results = sp.audio_analysis(track)['track']
    except:
      analysis_results = None

    if analysis_results == None:
        None_counter = None_counter + 1
    else:
        track_id.append(track)
        key_confidence.append(analysis_results['key_confidence'])
        mode_confidence.append(analysis_results['mode_confidence'])
        tempo_confidence.append(analysis_results['tempo_confidence'])
        time_signature_confidence.append(analysis_results['time_signature_confidence'])
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

100%|██████████| 9547/9547 [1:36:07<00:00,  1.66it/s]

Number of tracks where no audio features were available: 18
Time to run this code (in seconds): 5767.7546839609995


In [ ]:
df_analysis = pd.DataFrame({'track_id':track_id,'key_confidence':key_confidence,
                            'mode_confidence':mode_confidence,'tempo_confidence':tempo_confidence,
                            'time_signature_confidence':time_signature_confidence})
print(df_analysis.shape)
df_analysis.head()

(9529, 5)


,track_id,key_confidence,mode_confidence,tempo_confidence,time_signature_confidence
0,60wwxj6Dd9NJlirf84wr2c,0.592,0.700,0.778,0.582
1,2QtJA4gbwe1AcanB2p21aP,0.395,0.646,0.720,1.000
2,1dFkD1JfRMzwO6hwUsE8aS,0.299,0.477,0.764,0.646
3,3Gwq0zKnMWluOEPKh93BeC,0.478,0.603,0.708,0.987
4,5qA35ZZWFOqynheP5x8rty,0.383,0.486,0.927,1.000


In [ ]:
df = pd.merge(df,df_analysis,on='track_id',how='left')
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (9547, 21)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,key_confidence,mode_confidence,tempo_confidence,time_signature_confidence
0,Zedd,Clarity,60wwxj6Dd9NJlirf84wr2c,74,0.509,0.781,8,-3.480,1,0.0720,...,0.00000,0.0749,0.176,128.000,271427,4,0.592,0.700,0.778,0.582
1,Zedd,Stay The Night - Featuring Hayley Williams Of ...,2QtJA4gbwe1AcanB2p21aP,63,0.596,0.738,8,-3.109,1,0.0411,...,0.00000,0.0947,0.461,127.961,217347,4,0.395,0.646,0.720,1.000
2,Zedd,Spectrum,1dFkD1JfRMzwO6hwUsE8aS,59,0.609,0.801,0,-4.417,0,0.0340,...,0.00349,0.1270,0.407,129.990,243413,4,0.299,0.477,0.764,0.646
3,Zedd,Lost At Sea,3Gwq0zKnMWluOEPKh93BeC,47,0.567,0.796,6,-3.534,1,0.0364,...,0.00000,0.1240,0.305,127.983,225840,4,0.478,0.603,0.708,0.987
4,Noir,Around - Solomun Vox,5qA35ZZWFOqynheP5x8rty,56,0.867,0.345,1,-11.042,1,0.0768,...,0.66600,0.1030,0.259,115.000,418594,4,0.383,0.486,0.927,1.000


Everything seems in order so we save it to csv

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Spotify DA/Data/SpotifyAudioFeatures05012022.csv')